In [ ]:
!nvidia-smi

Sun Apr 11 11:45:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastcore onnx onnxruntime sentencepiece seqeval rouge-score
    !pip install -Uqq --no-deps fastai ohmeow-blurr
    !pip install -Uqq transformers datasets wandb 

     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 14.5MB 217kB/s 
     |████████████████████████████████| 4.1MB 55.4MB/s 
     |████████████████████████████████| 1.2MB 49.4MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 194kB 20.2MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 2.2MB 20.6MB/s 
     |████████████████████████████████| 194kB 53.7MB/s 
     |████████████████████████████████| 2.1MB 50.5MB/s 
     |████████████████████████████████| 870kB 55.3MB/s 
     |████████████████████████████████| 3.3MB 51.9MB/s 
     |████████████████████████████████| 112kB 61.0MB/s 
     |████████████████████████████████| 245kB 48.8MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 133kB 58.4MB/s 
     |████████████████████████████████| 163kB 62.7MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 

In [ ]:
from fastai.text.all import *
from fastai.callback.wandb import *

In [ ]:
from transformers import *
from datasets import load_dataset, concatenate_datasets

from blurr.data.all import *
from blurr.modeling.all import *

## Data preprocessing

In [ ]:
ds_name = 'snli'

In [ ]:
train_ds = load_dataset(ds_name, split='train')
valid_ds = load_dataset(ds_name, split='validation')

Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)



Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c. Subsequent calls will reuse this data.


In [ ]:
len(train_ds), len(valid_ds)

(550152, 10000)

In [ ]:
train_ds.column_names

['premise', 'hypothesis', 'label']

In [ ]:
train_ds[2]

{'hypothesis': 'A person is outdoors, on a horse.',
 'label': 0,
 'premise': 'A person on a horse jumps over a broken down airplane.'}

In [ ]:
train_ds = train_ds.select(range(1000))

In [ ]:
from collections import Counter

In [ ]:
Counter(train_ds['label'])

Counter({-1: 2, 0: 334, 1: 332, 2: 332})

In [ ]:
train_ds = train_ds.filter(lambda sample: sample['label'] in [0,1,2])
valid_ds = valid_ds.filter(lambda sample: sample['label'] in [0,1,2])

## Setup

In [ ]:
model_name = 'distilbert-base-uncased'

max_len = 512
bs = 16
val_bs = bs*2

## Tracking

In [ ]:
import wandb

WANDB_NAME = f'{ds_name}-{model_name}-simple'
GROUP = f'{ds_name}-{model_name}-simple'
NOTES = f'Simple finetuning {model_name} with RAdam lr=2e-5'
CONFIG = {}
TAGS =[model_name,ds_name,'radam','simple']

In [ ]:
wandb.init(reinit=True, project="vat", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Training

In [ ]:
def _to_device(e, device):
    if hasattr(e, 'to'): return e.to(device)
    elif isinstance(e, dict):
        for _, v in e.items():
            if hasattr(v, 'to'): v.to(device)
        return {k:(v.to(device) if hasattr(v, 'to') else v) for k, v in e.items()}

In [ ]:
@patch
def one_batch(self:Learner, i, b):
        self.iter = i
        b_on_device = tuple(_to_device(e, self.dls.device) for e in b) if self.dls.device is not None else b
        self._split(b_on_device)
        self._with_events(self._do_one_batch, 'batch', CancelBatchException)

In [ ]:
task = HF_TASKS_ALL.SequenceClassification
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, model_cls=AutoModelForSequenceClassification, tokenizer_cls=AutoTokenizer, 
                                                                               config_kwargs={'num_labels':3}, tokenizer_kwargs={'max_len':512})

In [ ]:
def get_x(sample):
    return sample['premise'], sample['hypothesis']

In [ ]:
ds = concatenate_datasets([train_ds, valid_ds])

In [ ]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model),
          CategoryBlock(vocab={0:'entailment', 1:'neutral', 2:'contradiction'}))
dblock = DataBlock(blocks=blocks,
                   get_x = get_x,
                   get_y=ItemGetter('label'),
                   splitter=IndexSplitter(list(range(len(train_ds), len(train_ds)+len(valid_ds)))))
# dblock.summary(train_ds)

In [ ]:
dls = dblock.dataloaders(ds, bs=bs, val_bs=val_bs)

In [ ]:
dls.one_batch()

({'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
model = HF_BaseModelWrapper(hf_model)
learn = Learner(dls,
                model,
                opt_func=RAdam,
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

# learn.blurr_summary()

### simple finetuning

In [ ]:
learn.fit_one_cycle(4, 3e-5, cbs=WandbCallback(log_preds=False, log_model=False))

Could not gather input dimensions


epoch,train_loss,valid_loss,accuracy,time
0,0.415137,0.341215,0.871266,39:57
1,0.320737,0.299613,0.893213,40:26
2,0.253862,0.289267,0.902154,41:19
3,0.171095,0.327337,0.900833,41:48


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


In [ ]:
learn.validate()

(#2) [0.32733726501464844,0.9008331894874573]

In [ ]:
test_ds = load_dataset('snli', split='test')
test_ds[0]

Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)


{'hypothesis': 'The church has cracks in the ceiling.',
 'label': 1,
 'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.'}

In [ ]:
test_ds = test_ds.filter(lambda s: s['label'] in [0,1,2])
test_dl = dls.test_dl(test_ds, with_labels=True)
learn.validate(dl=test_dl)

(#2) [0.3546602129936218,0.8931189179420471]

## Validation on adversarial data

In [ ]:
adv_ds = load_dataset('anli', split='test_r1')
adv_ds[0]

Reusing dataset anli (/root/.cache/huggingface/datasets/anli/plain_text/0.1.0/43fa2c99c10bf8478f1fa0860f7b122c6b277c4c41306255b7641257cf4e3299)


{'hypothesis': 'The first Ernest Jones store was opened on the continent of Europe.',
 'label': 0,
 'premise': 'Ernest Jones is a British jeweller and watchmaker. Established in 1949, its first store was opened in Oxford Street, London. Ernest Jones specialises in diamonds and watches, stocking brands such as Gucci and Emporio Armani. Ernest Jones is part of the Signet Jewelers group.',
 'reason': "The first store was opened in London, which is in Europe. It may have been difficult for the system because continents weren't mentioned.",
 'uid': '4aae63a8-fcf7-406c-a2f3-50c31c5934a9'}

In [ ]:
test_dl = dls.test_dl(adv_ds, with_labels=True)

In [ ]:
learn.validate(dl=test_dl)

(#2) [2.4942946434020996,0.29100000858306885]